In [1]:
#!gdown --id 1HV6zeS_pqNrDOfHr3A7UvJKnyzy5u4K1
#!gdown --id 1S6Eg67vDSUSx6usqnAYgHCR2oEYeNvmI

In [2]:
!nvidia-smi

Sat Oct 23 11:36:44 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 496.13       Driver Version: 496.13       CUDA Version: 11.5     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P8    12W /  N/A |    125MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 4GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [4]:
import math
import pandas as pd
import re
import numpy as np
import itertools
import collections
from collections import defaultdict, Counter
import string
from sklearn.preprocessing import LabelEncoder


import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import tensorflow_text as text
from bert import bert_tokenization
import numpy as np
from scipy.spatial import distance

import seaborn as sns
from sklearn.manifold import TSNE
import matplotlib as mpl
import matplotlib.pyplot as plt 
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [5]:
ls

 Volume in drive C is Acer
 Volume Serial Number is 3818-7542

 Directory of C:\Users\phvpa\OneDrive - Amrita Vishwa Vidyapeetham\Amrita-Mtech-CEN\ProjectWork\MTech Thesis\Denoise in Code mix data\Experiments\Malyalam

23-10-2021  11:29    <DIR>          .
23-10-2021  09:00    <DIR>          ..
23-10-2021  09:19    <DIR>          .ipynb_checkpoints
23-10-2021  11:33    <DIR>          1000 and 100 data
23-10-2021  11:29            51,833 1000and100_MuRIL_SVM_DT_RF.ipynb
20-10-2021  19:22            60,830 250and20MuRIL_SVM_DT_RF.ipynb
21-10-2021  01:01    <DIR>          Colab_work
23-10-2021  01:20            13,593 FastText.ipynb
23-10-2021  09:49    <DIR>          Indice_BERT
22-10-2021  04:02           187,035 Indice_BERT.ipynb
23-05-2021  09:42           268,731 Mal_sentiment_full_dev.tsv
16-06-2021  04:34           287,576 Mal_sentiment_full_test_withoutlabels.tsv
23-05-2021  09:41         2,382,197 Mal_sentiment_full_train.tsv
23-10-2021  09:17         6,268,128 Mal_train.npy
23-1

In [6]:
#train_df = pd.read_csv("Mal_sentiment_full_train.tsv", header=None, sep='\t')
train_df = pd.read_csv("Mal_sentiment_full_train.tsv", sep='\t')
print(len(train_df))

#test_df = pd.read_csv("Mal_sentiment_full_dev.tsv", header=None, sep='\t')
valid_df = pd.read_csv("Mal_sentiment_full_dev.tsv", sep='\t')
print(len(valid_df))

test_df = pd.read_csv("Mal_sentiment_full_test_withoutlabels.tsv", sep='\t')
len(test_df)

15888
1766


1962

In [7]:
train_df

,text,category
0,Sunny chechi fans evide like adichu power kani...,unknown_state
1,Angane july month ile ende aadyathe leave njan...,Positive
2,ഏട്ടന്റെ പുതിയ പടത്തിനു വേണ്ടി കാത്തിരിക്കുന്ന...,unknown_state
3,ഇനി ലാലേട്ടന്റെ വേട്ട തുടങ്ങാൻ പോകുന്നു..........,Positive
4,Trailer powli oru raksha illa . Pakshea padam ...,Positive
...,...,...
15883,Minimum 10 thavana kandavar maathram like adi,unknown_state
15884,Please cinema kaannaaan kothi aakunnu onnu rel...,Positive
15885,USA ok.. India No... Sadhacharam,unknown_state
15886,nama hero december 12 wait & see mamamaga mah...,Positive


In [8]:
train_df['category']

0         unknown_state
1              Positive
2         unknown_state
3              Positive
4              Positive
              ...      
15883     unknown_state
15884          Positive
15885     unknown_state
15886          Positive
15887    Mixed_feelings
Name: category, Length: 15888, dtype: object

In [9]:
#removing all the emoji from the given data

def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642"
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
        "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)


for i in range(len(train_df)):
    temp1 = deEmojify(train_df['text'][i])
    train_df['text'][i] = temp1

for j in range(len(valid_df)):
    temp2 = deEmojify(valid_df['text'][j])
    valid_df['text'][j] = temp2

for k in range(len(test_df)):
    temp3 = deEmojify(test_df['text'][k])
    test_df['text'][k]=temp3

In [10]:
def preprocess_data(text):
  user_name = '@[A-Za-z0-9]+' # Pattern for matching the user names in the tweet
  has_tag = '#[A-Za-z0-9]+' # Pattern for matching the hasgtags in the tweer
  url = 'https?:\/\/[A-Za-z0-9\.\/\-]+' # Pattern for matching the URLs in the tweet
  #emoji_pattern = r'/[x{1F600}-x{1F64F}]/u'
  
  x = text.lower()  # Lower casing all the characters
  x = re.sub(user_name, '', x) # Replace the username with an empty characrer
  x = re.sub(has_tag,'',x) # Replace the hashtags with an empty characrer
  x = re.sub(url, '', x) # Replace the URLs with an empty characrer
  #x = re.sub(emoji_pattern, '', x)
  x = x.translate(str.maketrans('', '', string.punctuation)) # Replace all the characters except alphabets and digits from the tweet

  return x

In [11]:
train_df['text'] = train_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
valid_df['text'] = valid_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe
test_df['text'] = test_df['text'].apply(lambda x: preprocess_data(x)) # Apply the clean_data() to all the entries in the dataframe

In [12]:
print(test_df['text'][279])
print(test_df['text'][516])
print(test_df['text'][546])
print(test_df['text'][555])
print(test_df['text'][559])
print(test_df['text'][570])
print(test_df['text'][825])
print(test_df['text'][838])
print(test_df['text'][860])
print(test_df['text'][890])
print(test_df['text'][1234])

oru panipalal manakunnu again mr jr 
ende ponno 68 vayasaya oraluda body thanneyano ith  namichu ponnikkaaaa
fdfs trailer kandu ithu vareyum njanithu paranjittilla
degraders step back   മമ്മൂക്ക
ഈ പാട്ട് ഡിസ്‌ലൈക് ചെയ്തവരുടെ വീട്ടുകാരെ സമ്മതിക്കണം big salute
ഇത് ഏത് ഭാഷ ശിവനെ ഏത് ജില്ല 
ഇതൊക്കെ മമ്മുക്ക പറഞ്ഞാൽ പൊളിക്കും   
108 st pius x kuttikanam        uyir
ente ponno heavy bgm ikka pinne parayanilla2 looks kidilosky  overall kidukkachi aahn mone
song dislike adicha 90 perum phycho singles akum
അഭ്യർത്ഥന  മാമാങ്കം പലകുറി കൊണ്ടാടും  എന്ന ഗാനം ടൈറ്റിൽ സോങ്ങ് ആയി ഉൾപ്പെടുത്താമോ  album vasantha geethangal singer k j yesudas


In [13]:
valid_df['text'][0]

'mammooka ninghal mass aa pwoli item'

In [14]:
print(train_df)
print()
print(valid_df)
print()
print(test_df)

                                                    text        category
0       sunny chechi fans evide like adichu power kaniku   unknown_state
1      angane july month ile ende aadyathe leave njan...        Positive
2      ഏട്ടന്റെ പുതിയ പടത്തിനു വേണ്ടി കാത്തിരിക്കുന്ന...   unknown_state
3      ഇനി ലാലേട്ടന്റെ വേട്ട തുടങ്ങാൻ പോകുന്നു മുൻ കാ...        Positive
4      trailer powli oru raksha illa  pakshea padam i...        Positive
...                                                  ...             ...
15883      minimum 10 thavana kandavar maathram like adi   unknown_state
15884  please cinema kaannaaan kothi aakunnu onnu rel...        Positive
15885                        usa ok india no sadhacharam   unknown_state
15886  nama hero december 12 wait  see  mamamaga maha...        Positive
15887  madhuraraja 1 hour 40 k likes  lucifer 1 min 4...  Mixed_feelings

[15888 rows x 2 columns]

                                                   text       category
0                   mammoo

# Dividing train and valid separatly:

### Val data

In [15]:
df_temp2 = valid_df.sort_values('category')
df_temp2

,text,category
974,tovinok pakaram unni mukundan mathiyarunnu,Mixed_feelings
166,puthanpanam movie pole thonniyath enikk maathr...,Mixed_feelings
536,richie orma vanathu eniku mathram ano,Mixed_feelings
700,friendship dayikk oru cake medichu kodukk fresh,Mixed_feelings
1363,pulimurugan copy adi pole ind stund seen ellam,Mixed_feelings
...,...,...
436,tesla code um കണ്ടിട്ട് ഇറങ്ങിയിട്ടുണ്ട് കുറേ,unknown_state
1180,കമന്റ്‌ വായിക്കുന്ന ലാലേട്ടൻ ഫാൻസിന് ലൈക് അടിക...,unknown_state
1182,eth sajiv pillai visuals ane alle,unknown_state
1186,katha manasilay tv il varambam kanam,unknown_state


In [16]:
count2 = 20
val_Mixed_feelings = df_temp2[:count2]
val_Negative = df_temp2[110:110+count2]
val_Positive=df_temp2[350:350+count2]
val_not_malayalam = df_temp2[1060:1060+count2]
val_unknown_state = df_temp2[1190:1190+count2]

In [17]:
Val_frames = [val_Mixed_feelings, val_Negative, val_Positive, val_not_malayalam, val_unknown_state ]
Val_results = pd.concat(Val_frames)

In [18]:
Sub_val = Val_results.sample(frac = 1)

In [19]:
Sub_val_list = Sub_val['text'].tolist()
Sub_val_labels_list = Sub_val['category'].tolist()

In [20]:
le = LabelEncoder()
Y_val = le.fit_transform(Sub_val_labels_list)
Y_val

array([4, 1, 2, 2, 0, 0, 2, 3, 0, 0, 4, 4, 0, 4, 4, 2, 0, 3, 2, 3, 0, 1,
       1, 1, 4, 1, 4, 1, 2, 2, 4, 4, 1, 1, 3, 2, 0, 3, 3, 0, 2, 2, 3, 2,
       2, 0, 1, 4, 0, 4, 3, 4, 1, 4, 0, 1, 1, 3, 3, 3, 3, 0, 1, 2, 0, 1,
       0, 3, 0, 1, 4, 3, 2, 1, 0, 3, 3, 3, 2, 3, 0, 3, 3, 2, 4, 2, 1, 0,
       4, 2, 2, 4, 4, 1, 1, 1, 2, 4, 4, 0], dtype=int64)

### Train Data

In [21]:
df_temp = train_df.sort_values('category')
count = 200
Mixed_feelings = df_temp[:count]
Negative = df_temp[950:950+count]
Positive=df_temp[3040:3040+count]
not_malayalam = df_temp[9460:9460+count]
unknown_state = df_temp[10620:10620+count]

In [22]:
train_frames = [Mixed_feelings, Negative, Positive, not_malayalam, unknown_state ]

train_result = pd.concat(train_frames) #concat

In [23]:
Sub_train = train_result.sample(frac = 1) #shuffle

In [24]:
Sub_train_list = Sub_train['text'].tolist()
Sub_labels_list = Sub_train['category'].tolist()

In [25]:
len(Sub_train_list)

1000

In [26]:
le = LabelEncoder()
Y_train = le.fit_transform(Sub_labels_list)

In [27]:
len(Y_train)

1000

In [28]:
Mal_val = np.array(Sub_val_list)
np.save('Mal_val.npy',Mal_val)
Mal_val_label = np.array(Y_val)
np.save('Mal_val_label.npy',Mal_val_label)

In [29]:
temp1 =np.load('Mal_val.npy')
print(temp1.shape)
temp1 =np.load('Mal_val_label.npy')
print(temp1.shape)

(100,)
(100,)


In [30]:
Mal_train = np.array(Sub_train_list)
np.save('Mal_train.npy',Mal_train)
Mal_train_label = np.array(Y_train)
np.save('Mal_train_label.npy',Mal_train_label)

In [31]:
temp1 =np.load('Mal_train.npy')
print(temp1.shape)
temp1 =np.load('Mal_train_label.npy')
print(temp1.shape)

(1000,)
(1000,)
